In [1]:
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

In [2]:
import os
N_WORKERS = os.cpu_count()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
from comet_ml import Experiment

N_EPOCHS = 50

# Create an instance of the Experiment class
experiment = Experiment(
    project_name="ResNet50 Binary Classifier",  # Replace with your project name
    workspace="cristy17001"  # Replace with your workspace name
)

experiment.set_name("ResNet18_binary_classifier_focal_loss_1")
experiment.log_parameters({
    "model": "ResNet18 Focal Loss 1",
    "optimizer": "AdamW",
    "lr": 1e-4,
    "weight_decay": 1e-5,
    "loss_function": "FocalLoss",
    "scheduler": "ReduceLROnPlateau",
    "pretrained": True,
    "Patience": 2,
    "batch_size": 64,
    "epochs": N_EPOCHS,
})

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/cristy17001/resnet50-binary-classifier/00a54550998f457cb3eed868f8f4fe60



In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

#### Loading Data

In [5]:
class ImageFolderWithPaths(datasets.ImageFolder):
    def __getitem__(self, index):
        # standard behavior
        image, label = super().__getitem__(index)
        # get the path
        path = self.imgs[index][0]
        return image, label, path

In [6]:
train_dataset = ImageFolderWithPaths('split_by_presence/train', transform=transform)
val_dataset = ImageFolderWithPaths('split_by_presence/val', transform=transform)
test_dataset = ImageFolderWithPaths('split_by_presence/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)

### Training RESNET Models 

In [7]:
# Define the Model
model = models.resnet18(weights=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 1)
model = model.to(device)

C:\Users\Cristiano\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


### Training MOBILENET Models

In [8]:
# model = models.mobilenet_v3_large(weights=models.MobileNet_V3_Large_Weights.IMAGENET1K_V1)
# num_features = model.classifier[3].in_features
# model.classifier[3] = nn.Linear(num_features, 1)
# model = model.to(device)

### Training SqueezeNet Models

In [9]:
# model = models.squeezenet1_0(pretrained=True)
# model.classifier[1] = nn.Conv2d(512, 1, kernel_size=1)
# model.num_classes = 1

# class SqueezeNetBinary(nn.Module):
#     def __init__(self, base_model):
#         super().__init__()
#         self.base = base_model

#     def forward(self, x):
#         x = self.base(x)
#         return x.view(x.size(0), -1)  # Flatten to shape [batch_size, 1]

# model = SqueezeNetBinary(model).to(device)

In [10]:
from torchvision.ops import sigmoid_focal_loss

# Define loss function, optimizer and learning rate
def focal_loss(inputs, targets):
    return sigmoid_focal_loss(inputs, targets, alpha=0.25, gamma=2, reduction='mean')

criterion = focal_loss  # Use focal loss for binary classification
# criterion = nn.BCEWithLogitsLoss()  # Alternatively, use BCEWithLogitsLoss for binary classification
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.1, verbose=True)

C:\Users\Cristiano\AppData\Roaming\Python\Python312\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [11]:
from collections import defaultdict
from pathlib import Path
import numpy as np
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score, f1_score


def evaluate(model, dataloader, criterion):
    model.eval()
    val_loss = 0.0

    all_preds = []
    all_labels = []

    board_predictions = defaultdict(list)
    board_truths = defaultdict(list)

    with torch.no_grad():
        for inputs, labels, filenames in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device).float().unsqueeze(1)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            probs = torch.sigmoid(outputs)
            preds = (probs > 0.5).int().cpu().numpy()
            labels_np = labels.cpu().numpy().astype(int)

            all_preds.extend(preds.flatten())
            all_labels.extend(labels_np.flatten())

            for i in range(len(filenames)):
                filename = Path(filenames[i]).name
                board_id = "_".join(filename.split("_")[:2])
                board_predictions[board_id].append(preds[i][0])
                board_truths[board_id].append(labels_np[i][0])

    # Binary classification metrics
    val_accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, zero_division=0)
    recall = recall_score(all_labels, all_preds, zero_division=0)
    f1 = f1_score(all_labels, all_preds, zero_division=0)

    # Piece count metrics
    pred_counts = [sum(vals) for vals in board_predictions.values()]
    true_counts = [sum(vals) for vals in board_truths.values()]

    mse = mean_squared_error(true_counts, pred_counts)
    rmse = np.sqrt(mse)

    # Piece count accuracy: fraction of boards with exact count match
    exact_matches = sum([1 for p, t in zip(pred_counts, true_counts) if p == t]) # Joint count accuracy only correct if all board pieces are correctly predicted
    piece_count_accuracy = exact_matches / len(true_counts) if true_counts else 0.0

    avg_val_loss = val_loss / len(dataloader)

    return avg_val_loss, val_accuracy, precision, recall, f1, mse, rmse, piece_count_accuracy

In [12]:
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import torch

def train_model(model, train_loader, val_loader, optimizer, criterion, scheduler, num_epochs=10, save_best=True):
    best_val_acc = 0.0
    best_model_wts = model.state_dict()

    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}

    os.makedirs("checkpoints", exist_ok=True)

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch + 1}/{num_epochs}")
        model.train()
        train_loss, correct, total = 0.0, 0, 0

        for inputs, labels, _ in tqdm(train_loader, desc="Training"):
            inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            predictions = (torch.sigmoid(outputs) > 0.5).float()
            correct += (predictions == labels).sum().item()
            total += labels.size(0)

        avg_train_loss = train_loss / len(train_loader)
        train_accuracy = correct / total

        val_loss, val_accuracy, precision, recall, f1, mse, rmse, piece_count_acc = evaluate(model, val_loader, criterion)

        print(f"Train Loss: {avg_train_loss:.4f} | Train Acc: {train_accuracy:.4f}")
        print(f"Val   Loss: {val_loss:.4f} | Val   Acc: {val_accuracy:.4f}")
        print(f"Val   Precision: {precision:.4f} | Recall: {recall:.4f} | F1: {f1:.4f}")
        print(f"Piece Count MSE: {mse:.4f} | RMSE: {rmse:.4f} | Piece Count Acc: {piece_count_acc:.4f}")

        experiment.log_metrics({
            "train_loss": avg_train_loss,
            "train_accuracy": train_accuracy,
            "val_loss": val_loss,
            "val_accuracy": val_accuracy,
            "val_precision": precision,
            "val_recall": recall,
            "val_f1": f1,
            "val_piece_count_mse": mse,
            "val_piece_count_rmse": rmse,
            "val_piece_count_accuracy": piece_count_acc,
            "lr": optimizer.param_groups[0]['lr']
        }, epoch=epoch)

        history['train_loss'].append(avg_train_loss)
        history['val_loss'].append(val_loss)
        history['train_acc'].append(train_accuracy)
        history['val_acc'].append(val_accuracy)
        history.setdefault('val_mse', []).append(mse)
        history.setdefault('val_rmse', []).append(rmse)

        # Save checkpoint
        checkpoint_path = f"checkpoints/epoch_{epoch+1}.pt"
        torch.save(model.state_dict(), checkpoint_path)
        experiment.log_model(f"model_epoch_{epoch+1}", checkpoint_path)

        # Save best model
        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            best_model_wts = model.state_dict()
            torch.save(best_model_wts, "best_piece_classifier.pt")
            experiment.log_model("best_model", "best_piece_classifier.pt")

        # Step the scheduler with validation loss
        scheduler.step(val_loss)

    model.load_state_dict(best_model_wts)
    return model, history

### Evaluation of the Model

In [13]:
best_model, history = train_model(model, train_loader, val_loader, optimizer, criterion, scheduler, num_epochs=N_EPOCHS)

# Save the best model
torch.save(best_model.state_dict(), "best_piece_classifier.pt")


Epoch 1/50


Training: 100%|██████████| 1442/1442 [02:38<00:00,  9.07it/s]


Train Loss: 0.0014 | Train Acc: 0.9955
Val   Loss: 0.0010 | Val   Acc: 0.9938
Val   Precision: 0.9997 | Recall: 0.9788 | F1: 0.9891
Piece Count MSE: 0.9697 | RMSE: 0.9847 | Piece Count Acc: 0.7636

Epoch 2/50


Training: 100%|██████████| 1442/1442 [02:05<00:00, 11.50it/s]


Train Loss: 0.0004 | Train Acc: 0.9988
Val   Loss: 0.0001 | Val   Acc: 0.9998
Val   Precision: 0.9997 | Recall: 0.9995 | F1: 0.9996
Piece Count MSE: 0.0152 | RMSE: 0.1231 | Piece Count Acc: 0.9848

Epoch 3/50


...

Train Loss: 0.0000 | Train Acc: 1.0000
Val   Loss: 0.0001 | Val   Acc: 0.9997
Val   Precision: 0.9997 | Recall: 0.9993 | F1: 0.9995
Piece Count MSE: 0.0182 | RMSE: 0.1348 | Piece Count Acc: 0.9818

Epoch 39/50


Training: 100%|██████████| 1442/1442 [02:03<00:00, 11.69it/s]


Train Loss: 0.0000 | Train Acc: 1.0000
Val   Loss: 0.0001 | Val   Acc: 0.9997
Val   Precision: 0.9997 | Recall: 0.9992 | F1: 0.9994
Piece Count MSE: 0.0212 | RMSE: 0.1456 | Piece Count Acc: 0.9788

Epoch 40/50


Training:  27%|██▋       | 396/1442 [00:33<01:29, 11.69it/s]


KeyboardInterrupt: 

In [ ]:
# End the experiment
experiment.end()